### Import necessary packages

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
# List of paths
paths = [
    "/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/han/database_data_availability_202206",
    "/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/han/database_data_availability_202212",
    "/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/han/database_data_availability_202306",
    "/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/han/database_data_availability_202312",
    "/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/han/database_data_availability_202406",
    "/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/han/database_data_availability_202412"
]

# Initialize a list to store the results
results = []

# Loop through each path, read the parquet file, and calculate the required values
for path in paths:
    data_availability = pd.read_parquet(path)
    
    # Extract substring from the last letter to index 32
    filename = path[-33:]
    
    # Calculate the percentage of engines with time-series data
    percentage_engines = len(data_availability) / 759 * 100
    
    # Calculate mean and std for each column except 'product_id' and 'time'
    mean_values = data_availability.drop(columns=['product_id', 'engine_type', 'time']).mean()
    std_values = data_availability.drop(columns=['product_id', 'engine_type', 'time']).std()
    
    # Create a dictionary to store the results for the current file
    result = {
        'filename': filename,
        'percentage_engines': percentage_engines
    }

    # Add mean and std values to the result dictionary
    for column in mean_values.index:
        result[f'mean_{column}'] = mean_values[column]
        result[f'std_{column}'] = std_values[column]
    
    # Append the result dictionary to the results list
    results.append(result)

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

# Print the results DataFrame
results_df

In [ ]:
# Initialize a list to store the results
engine_type_availability = []

# Loop through each path, read the parquet file, and calculate the required values
for path in paths:
    data_availability = pd.read_parquet(path)

    # Extract substring from the last letter to index 32
    filename = path[-33:]

    # Group by engine type and calculate availability for each sensor data column
    sensor_data_columns = [col for col in data_availability.columns if col not in ['product_id', 'engine_type', 'time']]

    for engine_type, group in data_availability.groupby('engine_type'):
        sensor_data_availability = group[sensor_data_columns].mean()
        # Create a dictionary to store the results for the current file and engine type
        result = {
            'filename': filename,
            'engine_type': engine_type,
            }
        # Add sensor data availability as separate columns
        for column in sensor_data_columns:
            result[column] = sensor_data_availability[column]
        # Append the result dictionary to the engine type availability list
        engine_type_availability.append(result)

# Convert the engine type availability list to a DataFrame\n",
engine_type_availability_df = pd.DataFrame(engine_type_availability)

# Print the engine type availability DataFrame\n",
engine_type_availability_df

In [ ]:
# # Visualize the data availability results using seaborn and matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Manually average the mean for some sensor columns belonging to the same sensor type
sensor_groups_mean = {
    'Exhaust gas temperature after exhaust valve': ['mean_te_exh_cyl_out__0', 'mean_te_exh_cyl_out__1', 'mean_te_exh_cyl_out__2', 'mean_te_exh_cyl_out__3', 'mean_te_exh_cyl_out__4', 'mean_te_exh_cyl_out__5', 'mean_te_exh_cyl_out__6'],
    'dP air across scavenge air cooler': ['mean_pd_air_ic__0'],
    'Turbine back pressure': ['mean_pr_exh_turb_out__0', 'mean_pr_exh_turb_out__1', 'mean_pr_exh_turb_out__2', 'mean_pr_exh_turb_out__3'],
    'Scavenge air cooler air outlet temperature': ['mean_te_air_ic_out__0', 'mean_te_air_ic_out__1', 'mean_te_air_ic_out__2', 'mean_te_air_ic_out__3'],
    'Seawater temperature': ['mean_te_seawater'],
    'TC air intake temperature, sensor 1': ['mean_te_air_comp_in_a__0', 'mean_te_air_comp_in_a__1', 'mean_te_air_comp_in_a__2', 'mean_te_air_comp_in_a__3'],
    'TC air intake temperature, sensor 2': ['mean_te_air_comp_in_b__0', 'mean_te_air_comp_in_b__1', 'mean_te_air_comp_in_b__2', 'mean_te_air_comp_in_b__3'],
    'Turbocharger speed': ['mean_fr_tc__0', 'mean_fr_tc__1', 'mean_fr_tc__2', 'mean_fr_tc__3'],
    'Engine room ambient pressure': ['mean_pr_baro'],
    'Exhaust gas receiver pressure': ['mean_pr_exh_rec'],
    'ECS, scavenging air receiver pressure': ['pr_air_scav_ecs'],
    'Scavenge air receiver temperature': ['te_air_scav_rec'],
    'Exhaust gas temperature at turbine inlet': ['te_exh_turb_in__0', 'te_exh_turb_in__1', 'te_exh_turb_in__2', 'te_exh_turb_in__3'],
    'PMI, estimated engine load': ['re_eng_load'],
    }

for sensor_type, sensors in sensor_groups_mean.items():
    results_df[f'mean_{sensor_type}'] = results_df[sensors].mean(axis=1)

# Create a new DataFrame named results_bysensor to include filename, percentage_engines, and average availability for each sensor type
results_meanbysensor = results_df[['filename', 'percentage_engines'] + [f'mean_{sensor_type}' for sensor_type in sensor_groups_mean.keys()]]

# Plot percentage of engines with time-series data for each file
plt.figure(figsize=(12, 6))
sns.barplot(x='filename', y='percentage_engines', data=results_meanbysensor)
plt.xticks(rotation=90)
plt.title('Percentage of Engines with Time-Series Data')
plt.xlabel('Filename')
plt.ylabel('Percentage of Engines(%)')
plt.show()

# Plot mean values for each sensor type for each file
mean_columns = [col for col in results_meanbysensor.columns if col.startswith('mean_')]
mean_df = results_meanbysensor[['filename'] + mean_columns].melt(id_vars='filename', var_name='sensor_data_column', value_name='mean_value')

plt.figure(figsize=(12, 6))
sns.barplot(x='filename', y='mean_value', hue='sensor_data_column', palette='tab20', data=mean_df)
plt.xticks(rotation=90)
plt.title('Mean Missing Percent Values for Sensor Data from 2022 to 2024')
plt.xlabel('Filename')
plt.ylabel('Mean Missing Percent Value (%)')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

In [ ]:

# Manually average the mean for some sensor columns belonging to the same sensor type
sensor_groups_std = {
    'Exhaust gas temperature after exhaust valve': ['std_te_exh_cyl_out__0', 'std_te_exh_cyl_out__1', 'std_te_exh_cyl_out__2', 'std_te_exh_cyl_out__3', 'std_te_exh_cyl_out__4', 'std_te_exh_cyl_out__5', 'std_te_exh_cyl_out__6'],
    'dP air across scavenge air cooler': ['std_pd_air_ic__0'],
    'Turbine back pressure': ['std_pr_exh_turb_out__0', 'std_pr_exh_turb_out__1', 'std_pr_exh_turb_out__2', 'std_pr_exh_turb_out__3'],
    'Scavenge air cooler air outlet temperature': ['std_te_air_ic_out__0', 'std_te_air_ic_out__1', 'std_te_air_ic_out__2', 'std_te_air_ic_out__3'],
    'Seawater temperature': ['std_te_seawater'],
    'TC air intake temperature, sensor 1': ['std_te_air_comp_in_a__0', 'std_te_air_comp_in_a__1', 'std_te_air_comp_in_a__2', 'std_te_air_comp_in_a__3'],
    'TC air intake temperature, sensor 2': ['std_te_air_comp_in_b__0', 'std_te_air_comp_in_b__1', 'std_te_air_comp_in_b__2', 'std_te_air_comp_in_b__3'],
    'Turbocharger speed': ['std_fr_tc__0', 'std_fr_tc__1', 'std_fr_tc__2', 'std_fr_tc__3'],
    'Engine room ambient pressure': ['std_pr_baro'],
    'Exhaust gas receiver pressure': ['std_pr_exh_rec'],
    'ECS, scavenging air receiver pressure': ['std_pr_air_scav_ecs'],
    'Scavenge air receiver temperature': ['std_te_air_scav_rec'],
    'Exhaust gas temperature at turbine inlet': ['std_te_exh_turb_in__0', 'std_te_exh_turb_in__1', 'std_te_exh_turb_in__2', 'std_te_exh_turb_in__3'],
    'PMI, estimated engine load': ['std_re_eng_load'],
    }

for sensor_type, sensors in sensor_groups_std.items():
    results_df[f'std_{sensor_type}'] = results_df[sensors].mean(axis=1)

# Create a new DataFrame named results_bysensor to include filename, percentage_engines, and average availability for each sensor type
results_std_bysensor = results_df[['filename', 'percentage_engines'] + [f'std_{sensor_type}' for sensor_type in sensor_groups_std.keys()]]


# Plot std values for each sensor data column for each file\n",
std_columns = [col for col in results_std_bysensor.columns if col.startswith('std_')]
std_df = results_std_bysensor[['filename'] + std_columns].melt(id_vars='filename', var_name='sensor_data_column', value_name='std_value')

plt.figure(figsize=(12, 6))
sns.barplot(x='filename', y='std_value', hue='sensor_data_column', palette='tab20', data=std_df)
plt.xticks(rotation=90)
plt.title('Standard Deviation Values for Missing Percent for Sensor Data from 2022 to 2024')
plt.xlabel('Filename')
plt.ylabel('Standard Deviation Value for Missing Percent')
plt.legend(loc='upper right', bbox_to_anchor=(1.5, 1))
plt.show()

#### Data availability for new dataframes

In [ ]:
# List of paths
paths_new = [
    "/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/han/database_data_availability_202204sfoc",
    "/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/han/database_data_availability_202209sfoc",
    "/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/han/database_data_availability_202304sfoc",
    "/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/han/database_data_availability_202309sfoc",
    "/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/han/database_data_availability_202404sfoc",
    "/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/han/database_data_availability_202409sfoc"
]

# Initialize a list to store the results
results_new = []

# Loop through each path, read the parquet file, and calculate the required values
for path in paths_new:
    data_availability = pd.read_parquet(path)
    
    # Extract substring from the last letter to index 32
    filename = path[-33:]
    
    # Calculate the percentage of engines with time-series data
    percentage_engines = len(data_availability) / 759 * 100
    
    # Calculate mean and std for each column except 'product_id' and 'time'
    mean_values = data_availability.drop(columns=['product_id', 'engine_type', 'time']).mean()
    std_values = data_availability.drop(columns=['product_id', 'engine_type', 'time']).std()
    
    # Create a dictionary to store the results for the current file
    result = {
        'filename': filename,
        'percentage_engines': percentage_engines
    }

    # Add mean and std values to the result dictionary
    for column in mean_values.index:
        result[f'mean_{column}'] = mean_values[column]
        result[f'std_{column}'] = std_values[column]
    
    # Append the result dictionary to the results list
    results_new.append(result)

# Convert the results list to a DataFrame
results_df_new = pd.DataFrame(results_new)

# Print the results DataFrame
results_df_new

In [ ]:
# Initialize a list to store the results
engine_type_availability_new = []

# Loop through each path, read the parquet file, and calculate the required values
for path in paths_new:
    data_availability = pd.read_parquet(path)

    # Extract substring from the last letter to index 32
    filename = path[-33:]

    # Group by engine type and calculate availability for each sensor data column
    sensor_data_columns = [col for col in data_availability.columns if col not in ['product_id', 'engine_type', 'time']]

    for engine_type, group in data_availability.groupby('engine_type'):
        sensor_data_availability = group[sensor_data_columns].mean()
        # Create a dictionary to store the results for the current file and engine type
        result = {
            'filename': filename,
            'engine_type': engine_type,
            }
        # Add sensor data availability as separate columns
        for column in sensor_data_columns:
            result[column] = sensor_data_availability[column]
        # Append the result dictionary to the engine type availability list
        engine_type_availability_new.append(result)

# Convert the engine type availability list to a DataFrame
engine_type_availability_df_new = pd.DataFrame(engine_type_availability_new)

# Print the engine type availability DataFrame
engine_type_availability_df_new

In [ ]:
import re

# Define regex patterns for each dimension
patterns = {
    'stroke_bore_ratio': r'(G|S|L|K)',
    'piston_diameter_cm': r'(\d{2})',
    'engine_concept': r'(ME-C|ME-B|MC-C)',
    'mark': r'(\d{1,2}\.)',
    'dot_number': r'(\.\d+)',
    'fuel_injection_concept': r'-(LGIM|LGIP|LGIA|GIE|GI|GA)(?:-[A-Z]+)?',
    'emission_reduction': r'-(LPSCR|HPSCR|EGRBP|EGRTC|EcoEGR|TII|W)(?:-[A-Z]+)?'
}


# Function to extract dimensions from engine_type
def extract_dimensions(engine_type):
    dimensions = {}
    for key, pattern in patterns.items():
        match = re.search(pattern, engine_type)
        if match:
            dimensions[key] = match.group(1)
        else:
            dimensions[key] = None
    return dimensions


# Apply the function to extract dimensions and create new columns in the DataFrame
dimensions_df = engine_type_availability_df_new['engine_type'].apply(extract_dimensions).apply(pd.Series)

# Create a new DataFrame for each dimension and calculate the average availability
dimension_dfs = {}
for dimension in patterns.keys():
    temp_df = pd.concat([engine_type_availability_df_new[['filename']], dimensions_df[[dimension]], engine_type_availability_df_new[['te_exh_cyl_out__0', 'te_exh_cyl_out__1',
       'te_exh_cyl_out__2', 'te_exh_cyl_out__3', 'te_exh_cyl_out__4',
       'te_exh_cyl_out__5', 'te_exh_cyl_out__6', 'pd_air_ic__0',
       'pr_exh_turb_out__0', 'pr_exh_turb_out__1', 'pr_exh_turb_out__2',
       'pr_exh_turb_out__3', 'te_air_ic_out__0', 'te_air_ic_out__1',
       'te_air_ic_out__2', 'te_air_ic_out__3', 'te_seawater',
       'te_air_comp_in_a__0', 'te_air_comp_in_a__1', 'te_air_comp_in_a__2',
       'te_air_comp_in_a__3', 'te_air_comp_in_b__0', 'te_air_comp_in_b__1',
       'te_air_comp_in_b__2', 'te_air_comp_in_b__3', 'fr_tc__0', 'fr_tc__1',
       'fr_tc__2', 'fr_tc__3', 'pr_baro', 'pr_exh_rec', 'pr_air_scav_ecs', 'te_air_scav_rec', 'te_exh_turb_in__0', 'te_exh_turb_in__1', 'te_exh_turb_in__2', 'te_exh_turb_in__3', 're_eng_load']]], axis=1)
    grouped_df = temp_df.groupby(['filename', dimension]).mean().reset_index()
    dimension_dfs[dimension] = grouped_df

# Print the resulting DataFrames for each dimension
for dimension, df in dimension_dfs.items():
    print(f"DataFrame for {dimension}:")
    print(df)
    print("\n")

In [ ]:
# # Visualize the data availability results using seaborn and matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Manually average the mean for some sensor columns belonging to the same sensor type
sensor_groups_mean = {
    'Exhaust gas temperature after exhaust valve': ['mean_te_exh_cyl_out__0', 'mean_te_exh_cyl_out__1', 'mean_te_exh_cyl_out__2', 'mean_te_exh_cyl_out__3', 'mean_te_exh_cyl_out__4', 'mean_te_exh_cyl_out__5', 'mean_te_exh_cyl_out__6'],
    'dP air across scavenge air cooler': ['mean_pd_air_ic__0'],
    'Turbine back pressure': ['mean_pr_exh_turb_out__0', 'mean_pr_exh_turb_out__1', 'mean_pr_exh_turb_out__2', 'mean_pr_exh_turb_out__3'],
    'Scavenge air cooler air outlet temperature': ['mean_te_air_ic_out__0', 'mean_te_air_ic_out__1', 'mean_te_air_ic_out__2', 'mean_te_air_ic_out__3'],
    'Seawater temperature': ['mean_te_seawater'],
    'TC air intake temperature, sensor 1': ['mean_te_air_comp_in_a__0', 'mean_te_air_comp_in_a__1', 'mean_te_air_comp_in_a__2', 'mean_te_air_comp_in_a__3'],
    'TC air intake temperature, sensor 2': ['mean_te_air_comp_in_b__0', 'mean_te_air_comp_in_b__1', 'mean_te_air_comp_in_b__2', 'mean_te_air_comp_in_b__3'],
    'Turbocharger speed': ['mean_fr_tc__0', 'mean_fr_tc__1', 'mean_fr_tc__2', 'mean_fr_tc__3'],
    'Engine room ambient pressure': ['mean_pr_baro'],
    'Exhaust gas receiver pressure': ['mean_pr_exh_rec'],
    'ECS, scavenging air receiver pressure': ['mean_pr_air_scav_ecs'],
    'Scavenge air receiver temperature': ['mean_te_air_scav_rec'],
    'Exhaust gas temperature at turbine inlet': ['mean_te_exh_turb_in__0', 'mean_te_exh_turb_in__1', 'mean_te_exh_turb_in__2', 'mean_te_exh_turb_in__3'],
    'PMI, estimated engine load': ['mean_re_eng_load'],
    }
for sensor_type, sensors in sensor_groups_mean.items():
    results_df_new[f'mean_{sensor_type}'] = results_df_new[sensors].mean(axis=1)

# Create a new DataFrame named results_bysensor to include filename, percentage_engines, and average availability for each sensor type
results_meanbysensor = results_df_new[['filename', 'percentage_engines'] + [f'mean_{sensor_type}' for sensor_type in sensor_groups_mean.keys()]]

# Plot percentage of engines with time-series data for each file
plt.figure(figsize=(12, 6))
sns.barplot(x='filename', y='percentage_engines', data=results_meanbysensor)
plt.xticks(rotation=90)
plt.title('Percentage of Engines with Time-Series Data')
plt.xlabel('Filename')
plt.ylabel('Percentage of Engines(%)')
plt.show()

# Plot mean values for each sensor type for each file
mean_columns = [col for col in results_meanbysensor.columns if col.startswith('mean_')]
mean_df = results_meanbysensor[['filename'] + mean_columns].melt(id_vars='filename', var_name='sensor_data_column', value_name='mean_value')

plt.figure(figsize=(12, 6))
sns.barplot(x='filename', y='mean_value', hue='sensor_data_column', palette='tab20', data=mean_df)
plt.xticks(rotation=90)
plt.title('Mean Missing Percent Values for Sensor Data from 2022 to 2024')
plt.xlabel('Filename')
plt.ylabel('Mean Missing Percent Value (%)')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

In [ ]:

# Manually average the mean for some sensor columns belonging to the same sensor type
sensor_groups_std = {
    'Exhaust gas temperature after exhaust valve': ['std_te_exh_cyl_out__0', 'std_te_exh_cyl_out__1', 'std_te_exh_cyl_out__2', 'std_te_exh_cyl_out__3', 'std_te_exh_cyl_out__4', 'std_te_exh_cyl_out__5', 'std_te_exh_cyl_out__6'],
    'dP air across scavenge air cooler': ['std_pd_air_ic__0'],
    'Turbine back pressure': ['std_pr_exh_turb_out__0', 'std_pr_exh_turb_out__1', 'std_pr_exh_turb_out__2', 'std_pr_exh_turb_out__3'],
    'Scavenge air cooler air outlet temperature': ['std_te_air_ic_out__0', 'std_te_air_ic_out__1', 'std_te_air_ic_out__2', 'std_te_air_ic_out__3'],
    'Seawater temperature': ['std_te_seawater'],
    'TC air intake temperature, sensor 1': ['std_te_air_comp_in_a__0', 'std_te_air_comp_in_a__1', 'std_te_air_comp_in_a__2', 'std_te_air_comp_in_a__3'],
    'TC air intake temperature, sensor 2': ['std_te_air_comp_in_b__0', 'std_te_air_comp_in_b__1', 'std_te_air_comp_in_b__2', 'std_te_air_comp_in_b__3'],
    'Turbocharger speed': ['std_fr_tc__0', 'std_fr_tc__1', 'std_fr_tc__2', 'std_fr_tc__3'],
    'Engine room ambient pressure': ['std_pr_baro'],
    'Exhaust gas receiver pressure': ['std_pr_exh_rec'],
    'ECS, scavenging air receiver pressure': ['std_pr_air_scav_ecs'],
    'Scavenge air receiver temperature': ['std_te_air_scav_rec'],
    'Exhaust gas temperature at turbine inlet': ['std_te_exh_turb_in__0', 'std_te_exh_turb_in__1', 'std_te_exh_turb_in__2', 'std_te_exh_turb_in__3'],
    'PMI, estimated engine load': ['std_re_eng_load'],
    }

for sensor_type, sensors in sensor_groups_std.items():
    results_df_new[f'std_{sensor_type}'] = results_df_new[sensors].mean(axis=1)

# Create a new DataFrame named results_bysensor to include filename, percentage_engines, and average availability for each sensor type
results_std_bysensor = results_df_new[['filename', 'percentage_engines'] + [f'std_{sensor_type}' for sensor_type in sensor_groups_std.keys()]]


# Plot std values for each sensor data column for each file\n",
std_columns = [col for col in results_std_bysensor.columns if col.startswith('std_')]
std_df = results_std_bysensor[['filename'] + std_columns].melt(id_vars='filename', var_name='sensor_data_column', value_name='std_value')

plt.figure(figsize=(12, 6))
sns.barplot(x='filename', y='std_value', hue='sensor_data_column', palette='tab20', data=std_df)
plt.xticks(rotation=90)
plt.title('Standard Deviation Values for Missing Percent for Sensor Data from 2022 to 2024')
plt.xlabel('Filename')
plt.ylabel('Standard Deviation Value for Missing Percent')
plt.legend(loc='upper right', bbox_to_anchor=(1.5, 1))
plt.show()